# Loading data with DisjointTimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-08-26 09:14:50,394][wrapper_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:50,399][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:50,419][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:50,423][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 1249.57it/s]
[2025-08-26 09:14:50,660][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1251.70it/s]
[2025-08-26 09:14:50,759][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1100.50it/s]
[2025-08-26 09:14:50,814][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [175 286  20 355 385 ... 455 224 254 270 500], Length=274
        Val time series IDs: [416 499 537  12 344 ... 339 250 192 331 220], Length=109
        Test time series IDs: [278 144  40 137 306 ... 164 525 215 212 392], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
disjoint_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
disjoint_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-08-26 09:14:50,820][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:50,821][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:50,821][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:50,822][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:50,822][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
disjoint_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
disjoint_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-08-26 09:14:50,826][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:50,827][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:50,828][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:50,828][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:50,828][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:14:50,837][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 47.16it/s]


(274, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = disjoint_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:14:53,074][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 63.55it/s]


(109, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = disjoint_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:14:53,595][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 107.22it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:53,705][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:53,729][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:53,732][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2594.52it/s]
[2025-08-26 09:14:53,842][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 2984.30it/s]
[2025-08-26 09:14:53,882][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3850.21it/s]
[2025-08-26 09:14:53,900][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [200 448 513 409 175 ...  38  66 335 121  24], Length=274
        Val time series IDs: [442 271 167 491 416 ... 110  33  63 365 382], Length=109
        Test time series IDs: [281 155 338 364 309 ...  53 143 479 444 509], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-26 09:14:53,908][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 51.43it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = DisjointTimeBasedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:55,957][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:55,969][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:55,972][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 4/4 [00:00<00:00, 2652.52it/s]
[2025-08-26 09:14:55,976][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = disjoint_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-26 09:14:55,985][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 1856.08it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:56,058][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:56,129][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:56,132][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 7109.21it/s]
[2025-08-26 09:14:56,177][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 6599.29it/s]
[2025-08-26 09:14:56,197][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 6339.71it/s]
[2025-08-26 09:14:56,208][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [451 302 488  35 396 ... 443 432 482 456 166], Length=274
        Val time series IDs: [ 66 459 319 261 241 ... 379 244 412 462 370], Length=109
        Test time series IDs: [252 178 105 519 332 ... 321  99 140 139  47], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
   

In [15]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-26 09:14:56,216][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1208.99it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
disjoint_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
disjoint_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-26 09:14:56,625][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:56,626][disjoint_time_based_config][INFO] - all_batch_size adjusted to 25 as it should be greater than or equal to sliding_window_size + sliding_window_prediction_size.
[2025-08-26 09:14:56,626][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-26 09:14:56,627][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:56,627][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-26 09:14:56,628][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-26 09:14:56,628][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:56,635][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:56,654][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:56,658][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 8295.84it/s]
[2025-08-26 09:14:56,695][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 8070.38it/s]
[2025-08-26 09:14:56,714][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7715.63it/s]
[2025-08-26 09:14:56,723][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [ 85 213 440 527   6 ... 540  80 300  64 147], Length=274
        Val time series IDs: [187 546 391  35 492 ... 421  24 430 149 442], Length=109
        Test time series IDs: [296 255 110 160 352 ... 182 538 501 242  63], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Anomaly handler
  

In [18]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-26 09:14:56,731][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1290.07it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:57,117][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:57,136][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:57,139][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2613.21it/s]
[2025-08-26 09:14:57,249][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3203.06it/s]
[2025-08-26 09:14:57,288][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3855.65it/s]
[2025-08-26 09:14:57,304][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [466 359 156 187 486 ... 453 147 264 184  48], Length=274
        Val time series IDs: [ 19 520 414 150 284 ... 393 446  76 229  49], Length=109
        Test time series IDs: [352  84  12  54 193 ...  50 270 464 543 190], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = disjoint_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,466.0,0.0,54.0,72.0,3826.0,50.0,4.55,1.63,52.0,4.73,1.79,52.0,4.73,1.79,0.770020,0.660156,0.320068,0.330078,0.64,170.279999
1,466.0,1.0,52.0,73.0,3758.0,43.0,4.78,1.72,45.0,5.00,1.94,45.0,5.00,1.94,0.689941,0.620117,0.370117,0.389893,0.01,183.880005
2,466.0,2.0,40.0,54.0,2974.0,35.0,3.50,1.18,38.0,3.80,1.23,39.0,3.90,1.29,0.640137,0.569824,0.379883,0.370117,0.00,145.050003
3,466.0,3.0,23.0,34.0,1828.0,22.0,3.14,0.90,22.0,3.14,0.90,22.0,3.14,0.90,0.709961,0.660156,0.439941,0.429932,0.04,171.070007
4,466.0,4.0,21.0,33.0,1493.0,19.0,3.80,0.84,21.0,4.20,1.10,21.0,4.20,1.10,0.910156,0.870117,0.529785,0.529785,0.11,205.899994
5,466.0,5.0,28.0,36.0,1757.0,24.0,3.43,1.51,25.0,3.57,1.51,25.0,3.57,1.51,0.839844,0.799805,0.479980,0.469971,0.00,200.399994
6,466.0,6.0,14.0,23.0,1237.0,12.0,2.00,0.89,14.0,2.33,1.21,13.0,2.17,1.17,0.640137,0.600098,0.360107,0.330078,0.00,146.610001
7,466.0,7.0,23.0,33.0,1672.0,22.0,3.14,1.86,22.0,3.14,1.86,22.0,3.14,1.86,0.799805,0.720215,0.280029,0.260010,0.10,173.149994
8,466.0,8.0,25.0,36.0,1744.0,20.0,2.86,1.21,24.0,3.43,1.81,24.0,3.43,1.81,0.740234,0.709961,0.409912,0.399902,0.08,201.380005
9,466.0,9.0,37.0,54.0,2964.0,31.0,4.43,1.81,34.0,4.86,2.27,36.0,5.14,2.48,0.680176,0.560059,0.389893,0.389893,1.11,186.470001


In [21]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets   n_bytes  \
 0                     466.0      0.0     54.0       72.0    3826.0   
 1                     466.0      1.0     52.0       73.0    3758.0   
 2                     466.0      2.0     40.0       54.0    2974.0   
 3                     466.0      3.0     23.0       34.0    1828.0   
 4                     466.0      4.0     21.0       33.0    1493.0   
 ...                     ...      ...      ...        ...       ...   
 3354                  466.0   3354.0     23.0       46.0    4227.0   
 3355                  466.0   3355.0     12.0       18.0     820.0   
 3356                  466.0   3356.0     22.0       34.0    2468.0   
 3357                  466.0   3357.0     24.0       34.0    1776.0   
 3358                  466.0   3358.0     27.0      463.0  565703.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               50.0            4.55            1.63              52.0

#### Val set

- Affected by `val_workers`.

In [22]:
df = disjoint_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,19.0,3359.0,78.0,414.0,57237.0,54.0,4.91,2.02,41.0,3.73,1.79,71.0,6.45,4.18,0.489990,0.429932,0.449951,0.459961,33.48,137.600006
1,19.0,3360.0,102.0,801.0,75584.0,73.0,4.56,1.90,81.0,5.06,1.95,91.0,5.69,3.11,0.580078,0.540039,0.409912,0.459961,14.72,102.970001
2,19.0,3361.0,116.0,755.0,89456.0,74.0,4.62,1.86,82.0,5.12,2.13,97.0,6.06,2.57,0.660156,0.649902,0.350098,0.379883,17.65,112.620003
3,19.0,3362.0,95.0,688.0,86776.0,64.0,4.00,1.97,73.0,4.56,2.31,82.0,5.12,2.39,0.600098,0.580078,0.439941,0.479980,14.32,106.510002
4,19.0,3363.0,170.0,1146.0,152705.0,108.0,5.68,2.11,133.0,7.00,3.48,143.0,7.53,3.06,0.709961,0.689941,0.399902,0.439941,11.56,91.750000
5,19.0,3364.0,119.0,1211.0,172612.0,82.0,4.56,2.33,106.0,5.89,3.76,106.0,5.89,2.83,0.640137,0.629883,0.569824,0.629883,23.10,104.860001
6,19.0,3365.0,89.0,1027.0,136567.0,68.0,3.40,1.50,82.0,4.10,1.83,80.0,4.00,1.72,0.660156,0.640137,0.540039,0.569824,10.30,77.139999
7,19.0,3366.0,83.0,715.0,123899.0,61.0,3.39,1.46,75.0,4.17,2.12,73.0,4.06,1.83,0.799805,0.799805,0.560059,0.560059,12.08,90.290001
8,19.0,3367.0,73.0,610.0,92446.0,58.0,3.22,1.17,63.0,3.50,0.99,68.0,3.78,1.26,0.720215,0.709961,0.500000,0.549805,9.20,93.029999
9,19.0,3368.0,88.0,928.0,184655.0,67.0,3.72,1.93,71.0,3.94,2.18,78.0,4.33,2.35,0.580078,0.549805,0.589844,0.600098,22.40,104.839996


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets   n_bytes  \
 0                      19.0   3359.0     78.0      414.0   57237.0   
 1                      19.0   3360.0    102.0      801.0   75584.0   
 2                      19.0   3361.0    116.0      755.0   89456.0   
 3                      19.0   3362.0     95.0      688.0   86776.0   
 4                      19.0   3363.0    170.0     1146.0  152705.0   
 ...                     ...      ...      ...        ...       ...   
 2010                   19.0   5369.0     92.0      669.0  101102.0   
 2011                   19.0   5370.0     89.0      757.0   97407.0   
 2012                   19.0   5371.0     49.0      339.0   37779.0   
 2013                   19.0   5372.0     67.0      490.0   58375.0   
 2014                   19.0   5373.0    139.0     1029.0  145060.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0               54.0            4.91            2.02              41.0

#### Test set

- Affected by `test_workers`.

In [24]:
df = disjoint_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,352.0,5374.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
1,352.0,5375.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
2,352.0,5376.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
3,352.0,5377.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
4,352.0,5378.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
5,352.0,5379.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
6,352.0,5380.0,5.0,6.0,408.0,4.0,2.0,1.41,4.0,2.0,1.41,5.0,2.5,2.12,0.199951,0.140015,0.600098,0.620117,0.0,172.380005
7,352.0,5381.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
8,352.0,5382.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000
9,352.0,5383.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.500000,0.500000,0.500000,0.500000,0.0,0.000000


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets  n_bytes  \
 0                     352.0   5374.0      0.0        0.0      0.0   
 1                     352.0   5375.0      0.0        0.0      0.0   
 2                     352.0   5376.0      0.0        0.0      0.0   
 3                     352.0   5377.0      0.0        0.0      0.0   
 4                     352.0   5378.0      0.0        0.0      0.0   
 ...                     ...      ...      ...        ...      ...   
 1338                  352.0   6712.0      0.0        0.0      0.0   
 1339                  352.0   6713.0      0.0        0.0      0.0   
 1340                  352.0   6714.0      0.0        0.0      0.0   
 1341                  352.0   6715.0      0.0        0.0      0.0   
 1342                  352.0   6716.0      0.0        0.0      0.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0                0.0             0.0             0.0               0.0   
 1      

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:14:59,898][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:14:59,918][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:14:59,922][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2567.97it/s]
[2025-08-26 09:15:00,032][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3111.84it/s]
[2025-08-26 09:15:00,071][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3258.79it/s]
[2025-08-26 09:15:00,089][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [ 46 199 525 119 210 ... 282  93 425 512 102], Length=274
        Val time series IDs: [324 179 444 427 198 ... 220 153 163 109  27], Length=109
        Test time series IDs: [420  70 166 194 207 ... 236 218 340 177 422], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = disjoint_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = disjoint_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-26 09:15:00,525][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-08-26 09:15:00,552][disjoint_time_based_config][INFO] - Finalization and validation completed successfully.
[2025-08-26 09:15:00,557][cesnet_dataset][INFO] - Updating config for train set.
100%|██████████| 274/274 [00:00<00:00, 2200.98it/s]
[2025-08-26 09:15:00,687][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 2634.40it/s]
[2025-08-26 09:15:00,735][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 2219.53it/s]
[2025-08-26 09:15:00,762][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [240 345 408 276 520 ... 171 436 213 152  83], Length=274
        Val time series IDs: [366 336 356 116  76 ...   1  59 202 538 329], Length=109
        Test time series IDs: [154   2 406 509 546 ... 175 166 234 109 465], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)